In [1]:
import numpy as np
from math import pi
import PyCEGO

In [2]:
# Problem originally from Sandgren
allowable_wire_diameters= [
    0.009, 0.0095, 0.0104, 0.0118, 0.0128, 0.0132,
    0.014, 0.015, 0.0162, 0.0173, 0.018, 0.020,
    0.023, 0.025, 0.028, 0.032, 0.035, 0.041,
    0.047, 0.054, 0.063, 0.072, 0.080, 0.092,
    0.105, 0.120, 0.135, 0.148, 0.162, 0.177,
    0.192, 0.207, 0.225, 0.244, 0.263, 0.283,
    0.307, 0.331, 0.362, 0.394, 0.4375, 0.500
]

# Constants of the problem
F_max = 1000.0 # [lb] Maximum working load
S = 189000.0 # [psi] Allowable maximum shear stress
l_max = 14.0 # [in] Maximum free length
d_min = 0.2 # [in] Minimum wire diameter
D_max = 3.0 # [in] Maximum outer diameter of the spring
F_p = 300.0 # [lb] Preload compression force
sigma_pm = 6.0 # [in] Maximum deflection under preload
sigma_w = 1.25 # [in] Deflection from preload to full load
G = 11.5e6 # [] Shear modulus of the material

def CEGO_obj(x):
    return obj([x[0].as_int(),
                x[1].as_double(),
                allowable_wire_diameters[x[2].as_int()]])

def obj(x):
    # Unpack the inputs
    N,D,d = x
    
    # Intermediate terms
    C_f = (4*(D/d)-1)/(4*(D/d)-4)+0.615*d/D
    K = G*d**4/(8*N*D**3)
    sigma_p = F_p/K
    l_f = F_max/K + 1.05*(N+2)*d
    
    g = np.zeros((9,))
    g[1] = 8*C_f*F_max*D/(pi*d**3) - S
    g[2] = l_f - l_max 
    # g3, g4, and g5 are handled explicitly via bounds; will always be met
    g[3] = d_min - d # will always be met
    g[4] = D-D_max # will always be met
    g[5] = 3.0-D/d # will always be met
    g[6] = sigma_p-sigma_pm
    g[7] = sigma_p+(F_max-F_p)/K + 1.05*(N+2)*d - l_f
    g[8] = sigma_w - (F_max - F_p)/K
    
    s = np.ones_like(g)
    s[7] = 1e5
    s[8] = 1e5
    
    c = np.array([1 if g_i < 0 else 1+s_i*g_i for s_i,g_i in zip(s,g)])
    assert((c >= 1).all())
    
    f = pi**2*D*d**2*(N+2)/4

    return f*np.product(c**3)

print(obj([10, 1.180701, 0.283])) # Sandgren (see Lampinen)
print(obj([9, 1.2287, 0.283])) # Chen and Tsao (see Lampinen)
print(obj([9, 1.227411, 0.283])) # Wu and Chao (see Lampinen)
print(obj([9, 1.2230410, 0.283])) # Lampinen


221838.499745
2.6708602742
2.66805833809
2.68299959711


In [3]:
D = 3
Nwired = len(allowable_wire_diameters)
CEGO_bounds = [PyCEGO.Bound(1, int(l_max/d_min)), # N
               PyCEGO.Bound(3*d_min, D_max), # D
               PyCEGO.Bound(27, Nwired-1) # d (indices thereof)
               ]

for ocounter in range(5):
    layers = PyCEGO.NumberishLayers(CEGO_obj, D, D*20, 1, 3)
    layers.set_bounds(CEGO_bounds)
    layers.set_builtin_evolver(PyCEGO.BuiltinEvolvers.differential_evolution)

    objs = []
    for counter in range(1000):
        layers.do_generation()
        objective, coeffs = layers.get_best()
        if counter % 50 == 0:
            print(layers.print_diagnostics())
        objs.append(objective)

i: 0 best: 9.7811 c: 11, 1.964320, 39,  queue: 0
i: 50 best: 2.80822 c: 10, 1.184234, 35,  queue: 0
i: 100 best: 2.65857 c: 9, 1.223046, 35,  queue: 0
i: 150 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 200 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 250 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 300 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 350 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 400 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 450 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 500 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 550 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 600 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 650 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 700 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 750 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 800 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 850 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 900 best: 2.65856 c: 9, 1.223041, 35,  queue: 0
i: 950 best: 2.65856 c: 9, 1.2230

In [4]:
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of GlobalBestPSO
bounds = (np.array((1,3*d_min, 0.2)), np.array((l_max/d_min, D_max, 0.5)))
print(bounds)
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=3,
                                    options=options, bounds = bounds)

def pyswarms_obj(x):
    o = np.array([obj(x[i,:]) for i in range(x.shape[0])])
    return o.T

# Perform optimization
stats = optimizer.optimize(pyswarms_obj, iters=1000)

(array([ 1. ,  0.6,  0.2]), array([ 70. ,   3. ,   0.5]))


INFO:pyswarms.single.global_best:================================
Optimization finished!
Final cost: 2.6284
Best value: [5.7873866854791638, 1.5269078085362362, 0.29931279378577047]

